<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/11.%20Chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 11: Training Deep Neural Networks

Melatih jaringan saraf yang dalam (DNN) bisa menjadi sulit. Bab ini membahas masalah-masalah utama yang sering muncul dan menyajikan berbagai teknik untuk mengatasinya.

* **Masalah Gradien yang Lenyap/Meledak (*Vanishing/Exploding Gradients*)**: Selama *backpropagation*, gradien seringkali menjadi semakin kecil (lenyap) atau semakin besar (meledak) saat mengalir ke lapisan-lapisan bawah. Hal ini membuat lapisan-lapisan awal sangat sulit untuk dilatih. Beberapa solusi untuk masalah ini adalah:
    * **Inisialisasi Bobot yang Lebih Baik**: Menggunakan metode inisialisasi seperti **Glorot (Xavier)** atau **He initialization** dapat membantu menjaga sinyal (dan gradien) mengalir dengan baik di seluruh jaringan.
    * **Fungsi Aktivasi Non-Saturasi**: Menggunakan fungsi aktivasi seperti **ReLU** dan variannya (**Leaky ReLU, ELU, SELU**) membantu mengurangi masalah gradien yang lenyap karena mereka tidak jenuh pada nilai positif.
    * **Batch Normalization (BN)**: Menambahkan lapisan BN di antara lapisan-lapisan lain. BN menormalisasi input di setiap lapisan, yang secara signifikan menstabilkan dan mempercepat training.
    * **Gradient Clipping**: Membatasi nilai gradien agar tidak melebihi *threshold* tertentu, mencegah gradien meledak.

* **Menggunakan Kembali Lapisan Pretrained (*Transfer Learning*)**: Daripada melatih DNN dari awal, seringkali lebih baik menggunakan kembali lapisan-lapisan bawah dari jaringan yang sudah dilatih pada dataset besar yang serupa. Ini mempercepat training secara dramatis dan membutuhkan lebih sedikit data.
    * **Strategi**: "Bekukan" (*freeze*) lapisan-lapisan yang digunakan kembali pada awalnya, latih lapisan-lapisan atas yang baru, lalu "cairkan" (*unfreeze*) beberapa lapisan dan lanjutkan training dengan *learning rate* yang lebih rendah.

* **Optimizer yang Lebih Cepat**: Menggunakan optimizer yang lebih baik daripada SGD standar dapat memberikan peningkatan kecepatan yang signifikan. Optimizer yang populer meliputi:
    * **Momentum** dan **Nesterov Accelerated Gradient (NAG)**.
    * **Optimizer dengan *Adaptive Learning Rate*** seperti **AdaGrad**, **RMSProp**, dan yang paling populer, **Adam** dan **Nadam**.

* **Penjadwalan Learning Rate (*Learning Rate Scheduling*)**: Mengubah *learning rate* selama training dapat membantu konvergensi lebih cepat dan lebih baik daripada menggunakan *learning rate* konstan. Beberapa jadwal populer adalah *power scheduling*, *exponential scheduling*, dan **1cycle scheduling**.

* **Teknik Regularisasi untuk Menghindari Overfitting**:
    * **Regularisasi ℓ₁ dan ℓ₂**: Menambahkan penalti pada bobot model.
    * **Dropout**: Teknik yang sangat populer di mana neuron secara acak "dijatuhkan" (diabaikan) selama setiap langkah training. Ini memaksa jaringan menjadi lebih kuat dan tidak terlalu bergantung pada neuron tertentu.
    * **Max-Norm Regularization**: Membatasi norma ℓ₂ dari vektor bobot yang masuk ke setiap neuron.

### 1. Inisialisasi He, Aktivasi ELU, dan Batch Normalization
Berikut adalah contoh model yang menerapkan beberapa praktik terbaik untuk mengatasi gradien yang tidak stabil.

```python
from tensorflow import keras

# Contoh model dengan inisialisasi He, aktivasi ELU, dan Batch Normalization
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), # Menambahkan BN sebelum lapisan Dense
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()
```
**Catatan**: Urutan antara lapisan aktivasi dan Batch Normalization masih menjadi perdebatan. Anda bisa menempatkan BN sebelum atau sesudah fungsi aktivasi.

### 2. Gradient Clipping
Untuk menerapkan *gradient clipping*, Anda cukup mengaturnya pada optimizer saat membuat instance.

```python
# Membuat optimizer dengan gradient clipping
# clipvalue memotong setiap komponen gradien secara individual
# clipnorm memotong seluruh norma gradien jika melebihi threshold
optimizer = keras.optimizers.SGD(clipvalue=1.0)

# Kemudian kompilasi model dengan optimizer ini
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)
```

### 3. Transfer Learning Menggunakan Keras
Contoh ini menunjukkan cara menggunakan model Xception yang sudah dilatih pada ImageNet untuk tugas baru.

```python
import tensorflow_datasets as tfds

# Memuat model Xception pretrained tanpa lapisan atasnya
# weights="imagenet" akan mengunduh bobot yang telah dilatih
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)

# Menambahkan lapisan atas yang baru untuk tugas kita
# Misalkan kita punya 5 kelas
n_classes = 5
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

# 1. Bekukan lapisan-lapisan dasar agar tidak ikut terlatih pada awalnya
for layer in base_model.layers:
    layer.trainable = False

# 2. Kompilasi dan latih hanya lapisan atas yang baru
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# history = model.fit(train_set, epochs=5, validation_data=valid_set)

# 3. Setelah beberapa epoch, cairkan lapisan dasar dan lanjutkan training
#    dengan learning rate yang sangat kecil untuk fine-tuning
print("\nMelakukan fine-tuning...")
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# history = model.fit(train_set, epochs=5, validation_data=valid_set)
```
**Catatan**: Kode di atas hanya kerangka. Anda memerlukan `train_set` dan `valid_set` yang sesuai untuk menjalankannya.

### 4. Optimizer Lanjutan dan Penjadwalan Learning Rate
Menggunakan optimizer Adam dan jadwal *learning rate* eksponensial.

```python
# Menggunakan optimizer Adam (seringkali merupakan pilihan default yang baik)
# optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# Membuat fungsi jadwal learning rate
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

# Membuat callback untuk learning rate scheduler
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

# Callback ini akan dipanggil saat melatih model
# history = model.fit(X_train, y_train, ..., callbacks=[lr_scheduler])
```

### 5. Regularisasi dengan Dropout
Menambahkan lapisan `Dropout` adalah cara yang efektif untuk mencegah *overfitting*.

```python
# Menambahkan lapisan Dropout di antara lapisan Dense
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # Menjatuhkan 20% neuron secara acak
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# history = model.fit(...)
```
Dropout hanya aktif selama training dan dinonaktifkan selama inferensi (prediksi).

